In [1]:
import json, os
import pandas as pd
from nltk.corpus import stopwords
import numpy as np
from pymorphy2 import MorphAnalyzer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
morph = MorphAnalyzer()
stops = set(stopwords.words('russian'))
from string import punctuation
from nltk.corpus import stopwords
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

In [2]:
#data
PATH_TO_DATA = '../ru_kw_eval_datasets-master/data'
files = [os.path.join(PATH_TO_DATA, file) for file in os.listdir(PATH_TO_DATA)]
data = pd.concat([pd.read_json(file, lines=True) for file in files][:1], axis=0, ignore_index=True)

In [3]:
part_data = data[:300]

In [5]:
data.to_csv('articals.csv')

In [4]:
#evaluation
def evaluate(true_kws, predicted_kws):
    assert len(true_kws) == len(predicted_kws)
    
    precisions = []
    recalls = []
    f1s = []
    jaccards = []
    
    for i in range(len(true_kws)):
        true_kw = set(true_kws[i])
        predicted_kw = set(predicted_kws[i])
        
        tp = len(true_kw & predicted_kw)
        union = len(true_kw | predicted_kw)
        fp = len(predicted_kw - true_kw)
        fn = len(true_kw - predicted_kw)
        
        if (tp+fp) == 0:
            prec = 0
        else:
            prec = tp / (tp + fp)
        
        if (tp+fn) == 0:
            rec = 0
        else:
            rec = tp / (tp + fn)
        if (prec+rec) == 0:
            f1 = 0
        else:
            f1 = (2*(prec*rec))/(prec+rec)
            
        jac = tp / union
        
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
        jaccards.append(jac)
    print('Precision - ', round(np.mean(precisions), 2))
    print('Recall - ', round(np.mean(recalls), 2))
    print('F1 - ', round(np.mean(f1s), 2))
    print('Jaccard - ', round(np.mean(jaccards), 2))

In [5]:
#normalization 
def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return words

In [22]:
text_data = ''
for i in part_data.content.tolist():
    text_data+=i
    text_data+=' '
text_data = text_data.replace('\n', '. ')
text_data = text_data.replace('.. ', '. ')

In [23]:
text_data[:100]

'Неизвестные помещения в гробнице Тутанхамона. Самой обсуждаемой египетской историей 2016 года, котор'

In [24]:
from nltk.tokenize import sent_tokenize

sent_tokenize_list = sent_tokenize(text_data)
clear_sents=[]
for i in sent_tokenize_list:
    clear_sent = []
    clear_sent = normalize(i)
    clear_sents.append(clear_sent)

In [25]:
import gensim
simple_model = gensim.models.FastText(clear_sents, size=100, window=5,
                                      min_count=0, workers=2)

In [26]:
def word_to_vec(split_text):
    return [simple_model[i] for i in split_text]

part_data['norm_text'] = part_data.content.apply(normalize)

/home/semen/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [11]:
part_data.head()

,content,keywords,summary,title,url,norm_text
0,Неизвестные помещения в гробнице Тутанхамона\n...,"[без политики, египет, исследование, наука, ис...",О самых значимых открытиях в египетской археол...,Дневник строителя пирамиды и ноги красавицы Не...,https://russian.rt.com/science/article/340861-...,"[неизвестный, помещение, гробница, тутанхамон,..."
1,В Нижнем Новгороде ФСБ нейтрализовала преступн...,"[владимир путин, выборы президента россии 2018...",В Нижнем Новгороде нейтрализован сторонник ИГ*...,«Оказал сопротивление и был нейтрализован»: в ...,https://russian.rt.com/russia/article/476044-f...,"[нижний, новгород, фсб, нейтрализовать, престу..."
2,Американский лидер Дональд Трамп продолжает бо...,"[в мире, дональд трамп, сми, сша, телевидение,...",Президент США Дональд Трамп грозит отзывом лиц...,«За попытки дискредитировать и очернить»: Трам...,https://russian.rt.comhttps://russian.rt.com/w...,"[американский, лидер, дональд, трамп, продолжа..."
3,Украина столкнулась с новым скандалом при орга...,"[деньги, евровидение, киев, музыка, скандал, у...",Европейский вещательный союз недоволен темпами...,Песня прощается с тобой: почему Украина не усп...,https://russian.rt.com/ussr/article/336895-ukr...,"[украина, столкнуться, новый, скандал, организ..."
4,Комиссия Совфеда по защите госсуверенитета рек...,"[google, rt, youtube, в мире, россия, сми, сша...",Профильные ведомства должны подготовить предло...,«Активное давление»: Совфед рекомендовал разра...,https://russian.rt.com/world/article/436876-go...,"[комиссия, совфеда, защита, госсуверенитет, ре..."


In [28]:
len(set(part_data.norm_text.sum())), len(simple_model.wv.vocab)

(23081, 23073)

In [29]:
part_data['text_to_vec'] = part_data.norm_text.apply(word_to_vec)

/home/semen/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/home/semen/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [34]:
len(part_data.text_to_vec[:1].tolist()[0])

1149

In [36]:
len(part_data.norm_text[:1].tolist()[0])

1149

In [50]:
k = np.array(part_data.text_to_vec[:1][0])

In [79]:
import re
import numpy
from gensim.models import word2vec
from nltk.corpus import stopwords
from numpy import *

def find_kw(trainData):
    trainData = numpy.array(trainData)
    # calculate cosine similarity matrix to use in pagerank algorithm for dense matrix, it is not
    # fast for sparse matrix
    # sim_matrix = 1-pairwise_distances(trainData, metric="cosine")

    # similarity matrix, it is 30 times faster for sparse matrix
    # replace this with A.dot(A.T).todense() for sparse representation
    similarity = numpy.dot(trainData, trainData.T)

    # squared magnitude of preference vectors (number of occurrences)
    square_mag = numpy.diag(similarity)

    # inverse squared magnitude
    inv_square_mag = 1 / square_mag

    # if it doesn't occur, set it's inverse magnitude to zero (instead of inf)
    inv_square_mag[numpy.isinf(inv_square_mag)] = 0

    # inverse of the magnitude
    inv_mag = numpy.sqrt(inv_square_mag)

    # cosine similarity (elementwise multiply by inverse magnitudes)
    cosine = similarity * inv_mag
    cosine = cosine.T * inv_mag


    # pagerank powermethod
    def powerMethod(A, x0, m, iter):
        n = A.shape[1]
        delta = m * (array([1] * n, dtype='float64') / n)
        for i in range(iter):
            x0 = dot((1 - m), dot(A, x0)) + delta
        return x0


    n = cosine.shape[1]  # A is n x n
    m = 0.15
    x0 = [1] * n

    pagerank_values = powerMethod(cosine, x0, m, 130)

    srt = numpy.argsort(pagerank_values)
    a = srt[0:20]
    return a


In [81]:
ppp=part_data[:15]
ppp['kw'] = ppp.text_to_vec.apply(find_kw)

/home/semen/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [87]:
evaluate(ppp.keywords, all_key_words)

Precision -  0.03
Recall -  0.07
F1 -  0.04
Jaccard -  0.02


In [85]:
da =ppp.norm_text.tolist()
all_key_words=[]
for ind,i in enumerate(ppp.kw.tolist()):
    k_words=[]
    for j in i:
        k_words.append(da[ind][j])
    all_key_words.append(k_words)

In [86]:
len(all_key_words)

15

filter words only one vxojdenie